In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, BatchNormalization, Dropout, AveragePooling2D

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing CIFAR10 dataset and separating them

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


file_name = "/content/drive/My Drive/Courses/DeepLearning/HW02/Q03/datas/data_batch_{0}".format(1)
train = unpickle(file_name)
for i in range(2,6):
    file_name = "/content/drive/My Drive/Courses/DeepLearning/HW02/Q03/datas/data_batch_{0}".format(i)
    temp = unpickle(file_name)
    train[b'data'] = np.concatenate((train[b'data'], temp[b'data']))
    train[b'labels'] = np.concatenate((train[b'labels'], temp[b'labels']))   

indices = tf.range(start=0, limit=tf.shape(train[b'labels'])[0], dtype=tf.int32)
shuffled_indices = tf.random.shuffle(indices)

train[b'data'] = tf.gather(train[b'data'], shuffled_indices)
train[b'labels'] = tf.gather(train[b'labels'], shuffled_indices)

validation = {}
num_train = 40000
validation[b'data'] = train[b'data'][num_train:]
validation[b'labels'] = train[b'labels'][num_train:]

train[b'data'] = train[b'data'][0:num_train]
train[b'labels'] = train[b'labels'][0:num_train]

file_name = "/content/drive/My Drive/Courses/DeepLearning/HW02/Q03/datas/test_batch"
test = unpickle(file_name)


indices = tf.range(start=0, limit=tf.shape(test[b'labels'])[0], dtype=tf.int32)
shuffled_indices = tf.random.shuffle(indices)

test[b'data'] = tf.gather(test[b'data'], shuffled_indices)
test[b'labels'] = tf.gather(test[b'labels'], shuffled_indices)


num_classes = 10
feature_vector_length = 32*32*3
input_shape = (feature_vector_length,)
img_size_1D = int(np.sqrt(feature_vector_length/3))


X_train = train[b'data']/255
Y_train = tf.keras.utils.to_categorical(train[b'labels'], num_classes)

X_validation = validation[b'data']/255
Y_validation = tf.keras.utils.to_categorical(validation[b'labels'], num_classes)

X_test = test[b'data']/255
Y_test = tf.keras.utils.to_categorical(test[b'labels'], num_classes)


X_train_3D = np.reshape(X_train,(40000,img_size_1D, img_size_1D, 3))
X_validation_3D = np.reshape(X_validation,(10000, img_size_1D, img_size_1D, 3))
X_test_3D = np.reshape(X_test,(10000,img_size_1D, img_size_1D, 3))

In [3]:
X_train_3D.shape

(40000, 32, 32, 3)

### Implementing the Model

In [4]:
batch_size = 64
epochs_number = 40
learning_rate = 1e-3

model = tf.keras.models.Sequential()

model.add(Conv2D(96, (1,1), kernel_initializer='glorot_uniform', padding='same', name='conv1', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((3, 3)))

model.add(Conv2D(16, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire2_squeeze'))
model.add(Conv2D(64, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire2_conv1'))
model.add(Conv2D(64, (3,3), kernel_initializer='glorot_uniform', padding='same', name='fire2_conv2'))

model.add(Conv2D(16, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire3_squeeze'))
model.add(Conv2D(64, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire3_conv1'))
model.add(Conv2D(64, (3,3), kernel_initializer='glorot_uniform', padding='same', name='fire3_conv2'))

model.add(Conv2D(32, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire4_squeeze'))
model.add(Conv2D(128, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire4_conv1'))
model.add(Conv2D(128, (3,3), kernel_initializer='glorot_uniform', padding='same', name='fire4_conv2'))

model.add(Conv2D(32, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire5_squeeze'))
model.add(Conv2D(128, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire5_conv1'))
model.add(Conv2D(128, (3,3), kernel_initializer='glorot_uniform', padding='same', name='fire5_conv2'))

model.add(Conv2D(48, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire6_squeeze'))
model.add(Conv2D(192, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire6_conv1'))
model.add(Conv2D(192, (3,3), kernel_initializer='glorot_uniform', padding='same', name='fire6_conv2'))

model.add(Conv2D(48, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire7_squeeze'))
model.add(Conv2D(192, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire7_conv1'))
model.add(Conv2D(192, (3,3), kernel_initializer='glorot_uniform', padding='same', name='fire7_conv2'))

model.add(Conv2D(64, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire8_squeeze'))
model.add(Conv2D(256, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire8_conv1'))
model.add(Conv2D(256, (3,3), kernel_initializer='glorot_uniform', padding='same', name='fire8_conv2'))
model.add(MaxPooling2D((3, 3)))

model.add(Conv2D(64, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire9_squeeze'))
model.add(Conv2D(256, (1,1), kernel_initializer='glorot_uniform', padding='same', name='fire9_conv1'))
model.add(Conv2D(256, (3,3), kernel_initializer='glorot_uniform', padding='same', name='fire9_conv2'))
model.add(Dropout(0.4))
model.add(Conv2D(1000, (1,1), kernel_initializer='glorot_uniform', padding='valid', name='conv10'))
model.add(AveragePooling2D((3, 3)))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train_3D, Y_train, epochs=epochs_number, batch_size=batch_size, validation_data=(X_validation_3D, Y_validation))


Epoch 1/40
625/625 [==============================] - 54s 68ms/step - loss: 2.0541 - accuracy: 0.2518 - val_loss: 1.7878 - val_accuracy: 0.3182
Epoch 2/40
625/625 [==============================] - 40s 63ms/step - loss: 1.7722 - accuracy: 0.3547 - val_loss: 1.6528 - val_accuracy: 0.4088
Epoch 3/40
625/625 [==============================] - 42s 67ms/step - loss: 1.6624 - accuracy: 0.4031 - val_loss: 1.6654 - val_accuracy: 0.4075
Epoch 4/40
625/625 [==============================] - 42s 67ms/step - loss: 1.5979 - accuracy: 0.4298 - val_loss: 1.6098 - val_accuracy: 0.4253
Epoch 5/40
625/625 [==============================] - 42s 67ms/step - loss: 1.5558 - accuracy: 0.4467 - val_loss: 1.4693 - val_accuracy: 0.4748
Epoch 6/40
625/625 [==============================] - 40s 64ms/step - loss: 1.5016 - accuracy: 0.4678 - val_loss: 1.6113 - val_accuracy: 0.4506
Epoch 7/40
625/625 [==============================] - 40s 64ms/step - loss: 1.4718 - accuracy: 0.4793 - val_loss: 1.4228 - val_accuracy:

In [ ]:
epochs_number = 120
model.fit(X_train_3D, Y_train, epochs=epochs_number, batch_size=batch_size, validation_data=(X_validation_3D, Y_validation), initial_epoch=40)


Epoch 41/120
625/625 [==============================] - 39s 62ms/step - loss: 0.8983 - accuracy: 0.6822 - val_loss: 1.2533 - val_accuracy: 0.5768
Epoch 42/120
625/625 [==============================] - 39s 62ms/step - loss: 0.8883 - accuracy: 0.6844 - val_loss: 1.3543 - val_accuracy: 0.5519
Epoch 43/120
625/625 [==============================] - 39s 62ms/step - loss: 0.8747 - accuracy: 0.6904 - val_loss: 1.3228 - val_accuracy: 0.5781
Epoch 44/120
625/625 [==============================] - 41s 66ms/step - loss: 0.8704 - accuracy: 0.6925 - val_loss: 1.2963 - val_accuracy: 0.5762
Epoch 45/120
625/625 [==============================] - 39s 62ms/step - loss: 0.8589 - accuracy: 0.6938 - val_loss: 1.3397 - val_accuracy: 0.5787
Epoch 46/120
625/625 [==============================] - 39s 62ms/step - loss: 0.8519 - accuracy: 0.6958 - val_loss: 1.3246 - val_accuracy: 0.5775
Epoch 47/120
625/625 [==============================] - 39s 62ms/step - loss: 0.8431 - accuracy: 0.7011 - val_loss: 1.3455 -